In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
#torch.cuda.empty_cache()

In [2]:
import torch

device = (
    "mps"
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cuda


C:\Users\jigar\AppData\Local\Temp\ipykernel_8616\763796151.py:5: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  if getattr(torch, "has_mps", False)


In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.device_count()

1

In [5]:
torch.cuda.current_device()

0

In [6]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cuda



In [7]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

In [8]:
folder_name = 'sh06x2'
train_exe = 1920

train_x = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_train_x_m_{train_exe}.npy')[:768000]
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1],1)
train_y = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_train_y_m_{train_exe}.npy')[:768000]

#(1000/(1.6*256))*400
#379800-(200*633)
#976 mhz:768000 and 253200

val_x = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_val_x_m_{train_exe}.npy')[:253200]
val_x = val_x.reshape(val_x.shape[0], val_x.shape[1],1)
val_y = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_val_y_m_{train_exe}.npy')[:253200]

In [9]:
print(train_x.shape,train_y.shape)
print(val_x.shape, val_y.shape)

(1152000, 256, 1) (768000, 256)
(253200, 256, 1) (253200, 256)


In [8]:
379800/600

633.0

In [9]:
(1000/(1.6*256))*400

976.5625

In [10]:
379800-(200*633)

253200

In [11]:
train_x = torch.tensor(train_x, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)

val_x = torch.tensor(val_x, dtype=torch.float32)
val_y = torch.tensor(val_y, dtype=torch.float32)

In [12]:
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

torch.Size([768000, 256, 1]) torch.Size([768000, 256])
torch.Size([253200, 256, 1]) torch.Size([253200, 256])


In [13]:
# Setup data loaders for batch
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

val_dataset = TensorDataset(val_x, val_y)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

In [14]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    #def __init__(self, d_model, dropout=0.1, max_len=5000):
    def __init__(self, d_model, dropout=0.0, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [15]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=2, num_layers=2, dropout=0.1):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, 256)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.encoder(x)
        #print(f"encoder : {x.shape}")
        x = self.pos_encoder(x)
        #print(f"position encoder : {x.shape}")
        x = self.transformer_encoder(x)
        #print(f"transformer encoder : {x.shape}")
        x = self.decoder(x[:, -1, :])
        x = self.relu(x)
        #print(f"decoder : {x.shape}")
        return x

model = TransformerModel().to(device)
output = model(torch.randn(1,256,1).to(device))
total_params = sum(p.numel() for p in model.parameters())
print(f'Total Parameters: {total_params}')

C:\Users\jigar\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Total Parameters: 579072


In [16]:
# # Access parameters layer-by-layer
# for name, param in model.named_parameters():
#     print(f'Layer: {name}, Size: {param.size()}')

In [17]:
# from torchviz import make_dot
# x = torch.zeros(1, 256, 1, dtype=torch.float, requires_grad=False).to(device)
# out = model(x)
# make_dot(out)  # plot graph of variable, not of a nn.Module

In [18]:
from tqdm import tqdm
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 100
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    train_losses = []
    for batch in tqdm(train_loader, desc='Train : '):
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=' Val : '):
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    train_loss = np.mean(train_losses)
    scheduler.step(val_loss)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 5:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

 Val : 100%|█████████████████████████████████████████████████████████████████████████| 990/990 [01:08<00:00, 14.38it/s]


Epoch 1/100, Loss: 0.8551, Val Loss: 0.9694


 Val : 100%|█████████████████████████████████████████████████████████████████████████| 990/990 [01:09<00:00, 14.21it/s]


Epoch 2/100, Loss: 0.8453, Val Loss: 0.9768


 Val : 100%|█████████████████████████████████████████████████████████████████████████| 990/990 [01:09<00:00, 14.17it/s]


...

Epoch 11/100, Loss: 0.8339, Val Loss: 0.8487


 Val : 100%|█████████████████████████████████████████████████████████████████████████| 990/990 [01:09<00:00, 14.14it/s]


Epoch 12/100, Loss: 0.8338, Val Loss: 0.8448


Train :  16%|███████████▌                                                           | 488/3000 [01:48<09:19,  4.49it/s]


KeyboardInterrupt: 